In [116]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

In [117]:
from google.colab import drive
drive.mount('/content/drive')
mypath = '/content/drive/My Drive/hd trees/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import and Process Images

In [ ]:
# Import and Process Images
filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
images = []

# Define lower and uppper limits of what we call sky blue
sky_lo=np.array([128,0,0])
sky_hi=np.array([255,220,220])

# Iterate through folder
for i in range(len(filenames)):
  image = (cv2.imread(mypath+filenames[i],1))
  image = cv2.resize(image, None, fx=0.75, fy=0.75, interpolation=cv2.INTER_AREA)
  
  # Mask image to only select sky
  mask = cv2.inRange(image,sky_lo,sky_hi)

  # Change image to white where we found sky
  image[mask>0]=(255,255,255)

  # Threshold
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  thresh, image = cv2.threshold(gray, 128, 255, cv2.THRESH_OTSU)

  # Add image to array
  images.append(image)

  # Write to drive
  cv2.imwrite(f'{mypath}/processed/{filenames[i]}', image)

  print(f'{i/len(filenames):.2%}')

# Box Counting Function

In [120]:
def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

# Box Counting Results

In [121]:
# Thresholded Images
for i in range(len(images)):
  D = fractal_dimension(images[i])
  print(f'{filenames[i]}: D~{D:.2f}')

Cottonwoods.jpg: 1.86
Ash.jpg: 1.88
Eucalyptus.jpg: 1.86
Coral.jpg: 1.82
Camphor.jpg: 1.92
Tipu.jpg: 1.93
ScrubOaks.jpg: 1.81
MacDonaldOak.jpg: 1.89
IslandOaks.jpg: 1.87
Ironwoods.tif: 1.80
